<a href="https://colab.research.google.com/github/gauss5930/Deep-Learning-Paper/blob/main/Computer%20Vision/CNN/Xception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Xception 모델 설명
#당시, ImageNet에 대해서 SoTA를 차지함
#VGG16과 ResNet의 입력 이미지 크기(224x224)와 다르게 (299x299)를 사용함
#전처리 방식도 다름(Inception V3와 동일)

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import warnings

import keras
from keras import layers
from keras.models import Sequential
from keras import backend
#얘네는 오류 발생
#from . import get_submodules_from_kwargs
#from . import imagenet_utils
#from .imagenet_utils import decode_predictions
#from .imagenet_utils import _obtain_input_shape

TF_WEIGHTS_PATH = (
    'https://github.com/fchollet/deep-learning-models/'
    'releases/download/v0.4/'
    'xception_weights_tf_dim_ordering_tf_kernels.h5'
)

TF_WEIGHTS_PATH_NO_TOP = (
    'https://github.com/fchollet/deep-learning-models/'
    'releases/download/v0.4/'
    'xception_weights_tf_dim_ordering_tf_kernels_notop.h5'
)

def Xception(include_top = True, weights = 'imagenet', input_tensor = None, 
             input_shape = None, pooling = None, classes = 1000, **kwargs):
  
  #기본 입력 이미지의 크기는 299 x 299
  #include_top: network의 맨 위에서 fc-layer을 포함할 지
  #weights: 'None'은 무작위, 'imagenet'은 Imagenet에서 pre-training, 또는 업로드할 파일 경로
  #input_tensor: 모델의 입력 이미지에 대해 사용할 추가적인 keras tensor
  #input_shape: 옵션적 tuple 모양, 'include_top'이 False일 때만 사용 가능
  #pooling: feature extraction을 위한 옵션적 pooling mode, 'include_top'이 False일 때만 사용 가능
  #'None': 모델 출력이 4D tensor, 'avg': global average pooling이고 output은 2D tensor
  #'max': global max pooling
  #classes: 옵션적 class 수. 'include_top'이 True일 때와 'weights'가 명시되지 않았을 때 사용 가능

  #weights에 아무런 값이 없을 때
  if not (weights in {'imagenet', None} or os.path.exists(weights)):
    raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization), `imagenet` '
                         '(pre-training on ImageNet), '
                         'or the path to the weights file to be loaded.')
  
  #imagenet을 weights로 사용하는데 조건이 맞지 않을 때
  if weights == 'imagenet' and include_top and classes != 1000:
    raise ValueError('If using `weights` as `"imagenet"` with `include_top`'
                         ' as true, `classes` should be 1000')
    
  #적절한 입력 모양 결정
  input_shape = _obtain_input_shape(input_shape, default_size = 299, min_size = 71,
                                    data_format = backend.image_data_format(),
                                    require_flatten = include_top, weights = weights)
  
  if input_tensor is None:
    img_input = layers.Input(shape = input_shape)
  else:
    if not backend.is_keras_tensor(input_tensor):
      img_input = layers.Input(tensor = input_tensor, shape = input_shape)
    else:
      img_input = input_tensor

  channel_axis = 1 if backend.image_data_format() == 'channels_first' else -1

  #Entry Flow
  #입력 이미지 단계
  x = layers.Conv2D(32, (3, 3), strides = (2, 2), use_bias = False,
                    name = 'block1_conv1')(img_input)
  x = layers.BatchNormalization(axis = channel_axis, name = 'block1_conv1_bn')(x)
  x = layers.Activation('relu', name = 'block1_conv1_act')(x)
  x = layers.Conv2D(64, (3, 3), use_bias = False, name = 'block1_conv2_bn')(x)
  x = layers.Activation('relu', name = 'block1_conv2_act')(x)

  #첫 번째 residual network
  residual = layers.Conv2d(128, (1, 1), strides = (2, 2), padding = 'same', use_bias = False)(x)
  residual = layers.BatchNormalization(axis = channel_axis)(residual)

  x = layers.SeparableConv2D(128, (3, 3), padding = 'same', use_bias = False,
                             name = 'block2_sepconv1')(x)
  x = layers.BatchNormalization(axis = channel_axis, name = 'block2_sepconv1_bn')(x)
  x = layers.Activation('relu', name = 'block2_sepconv2_act')(x)
  x = layers.SeparableConv2D(128, (3, 3), padding = 'same', use_bias = 'same', 
                             name = 'block2_sepconv2')(x)
  x = layers.BatchNormalization(axis = channel_axis, name = 'block2_sepconv2_bn')(x)

  x = layers.MaxPooling2D((3, 3), strides = (2, 2), padding = 'same', 
                          name = 'block2_pool')(x)
  x = layers.add([x, residual])

  #두 번째 residual network
  residual = layers.Conv2d(256, (1, 1), strides = (2, 2), padding = 'same', use_bias = False)(x)
  residual = layers.BatchNormalization(sxis = channel_axis)(residual)

  x = layers.Activation('relu', name = 'block3_conv1_act')(x)
  x = layers.SeparableConv2D(256, (3, 3), strides = 'same', use_bias = False, 
                             name = 'block3_sepconv1')(x)
  x = layers.BatchNormalization(axis = channel_axis, name = 'block3_sepconv1_bn')(x)

  x = layers.Activation('relu', name = 'block3_conv2_act')(x)
  x = layers.SeparableConv2D(256, (3, 3), strides = 'same', use_bias = False, 
                             name = 'block3_sepconv2')(x)
  x = layers.BatchNormalization(axis = channel_axis, name = 'block3_sepconv2_bn')(x)

  x = layers.MaxPooling2D((3, 3), strides = (2, 2), padding = 'same', 
                          name = 'block3_pool')(x)

  x = layers.add([x, residual])

  #세 번째 residual network
  residual = layers.Conv2d(728, (1, 1), strides = (2, 2), padding = 'same', use_bias = False)(x)
  residual = layers.BatchNormalization(axis = channel_axis)(residual)

  x = layers.Activation('relu', name = 'block4_conv1_act')(x)
  x = layers.SeparableConv2D(728, (3, 3), strides = 'same', use_bias = False, 
                             name = 'block4_sepconv1')(x)
  x = layers.BatchNormalization(axis = channel_axis, name = 'block4_sepconv1_bn')(x)

  x = layer.Activation('relu', name = 'block4_conv2_act')(x)
  x = layers.SeparableConv2D(728, (3, 3), strides = 'same', use_bias = False,
                             name = 'block4_sepconv2')(x)
  x = layers.BatchNormalization(axis = channel_axis, name = 'block4_sepconv2_bn')(x)
  
  x = MaxPooling2D((3, 3), strides = (2, 2), padding = 'same', name = 'block4_pool')(x)

  x = layers.add([x, residual])

  #Middle Flow
  for i in range(8):
    residual = x
    prefix = 'block' + str(i + 5)   #블록 이름 지정 자동화
    
    x = layers.Activation('relu', name = prefix + '_sepconv1_act')(x)
    x = layers.SeparableConv2D(728, (3, 3), strides = 'same', use_bias = False, 
                               name = prefix + '_sepconv1')(x)
    x = layers.BatchNormalization(axis = channel_axis, name = prefix + '_sepconv1_bn')(x)

    x = layers.Activation('relu', name = prefix + '_sepconv2_act')(x)
    x = layers.SeparableConv2D(728, (3, 3), strides = 'same', use_bias = False, 
                               name = prefix + '_sepconv2')(x)
    x = layers.BatchNormalization(axis = channel_axis, name = prefix + '_sepconv2_bn')(x)

    x = layers.Activation('relu', name = prefix + '_sepconv3_act')(x)
    x = layers.SeparableConv2D(728, (3, 3), strides = 'same', use_bias = False, 
                               name = prefix + '_sepconv3')(x)
    x = layers.BatchNormalization(axis = channel_axis, name = prefix + '_sepconv3_bn')(x)

    x = layers.add([x, residual])

  #Exit Flow
  residual = layers.Conv2d(1024, (1, 1), strides = (2, 2), padding = 'same', use_bias = False)(x)
  residual = layers.BatchNormalization(axis = channel_axis)(residual)

  x = layers.Activation('relu', name = 'block13_sepconv1_act')(x)
  x = layers.SeparableConv2D(728, (3, 3), strides = 'same', use_bias = False, 
                             name = 'block13_sepconv1')(x)
  x = layers.BatchNormalization(axis = channel_axis, name = 'block13_sepconv1_bn')(x)

  x = layers.Activation('relu', name = 'block13_speconv2_act')(x)
  x = layers.SeparableConv2D(1024, (3, 3), strides = 'same', use_bias = False, 
                             name = 'block13_sepconv2')(x)
  x = layers.BatchNormalization(axis = channel_axis, name = 'block13_sepconv2_bn')(x)

  x = layers.MaxPooling2D((3, 3), strides = (2, 2), padding = 'same', name = 'block13_pool')(x)

  x = layers.add([x, residual])

  x = layers.SeparableConv2D(1536, (3, 3), strides = 'same', use_biad = False,
                             name = 'block14_sepconv1')(x)
  x = layers.BatchNormalization(axis = channel_axis, name = 'block14_sepconv1_bn')(x)
  x = layers.Activation('relu', name = 'block14_sepconv_act')(x)

  x = layers.SeparableConv2D(2048, (3, 3), strides = 'same', use_bias = False,
                             name = 'block14_sepconv2')(x)
  x = layers.BatchNormalization(axis = channel_axis, name = 'block14_speconv2_bn')(x)
  x = layers.Activation('relu', name = 'block14_sepconv2_act')(x)

  if include_top:
    x = layers.GlobalAveragePooling2D(name = 'avg_pool')(x)
    x = layers.Dense(classes, activation = 'softmax', name = 'predictions')(x)
  else:
    if pooling == 'avg':
      x = layers.GlobalAveragePooling2D()(x)
    elif pooling == 'max':
      x = layers.MaxPooling2D()(x)

  if input_tensor is not None:
    inputs = keras_utils.get_source_inputs(input_tensor)
  else:
    inputs = img_input

  #모델 생성
  if weights == 'imagenet':
    if include_top:
      weights_path = keras_utils.get_file(
          'xception_weights_tf_dim_ordering_tf_kernels.h5',
          TF_WEIGHTS_PATH,
          cache_subdir='models',
          file_hash='0a58e3b7378bc2990ea3b43d5981f1f6'
      )
    else:
      weights_path = keras_utils.get_file(
          'xception_weights_tf_dim_ordering_tf_kernels_notop.h5',
          TF_WEIGHTS_PATH_NO_TOP,
          cache_subdir='models',
          file_hash='b0042744bf5b25fce3cb969f33bebb97'
      )
    model.load_weights(weights_path)
    if backend.backend() == 'theano':
      keras_utils.convert_all_kernels_in_model(model)
  elif weights is not None:
    model.load_weights(weights)

  return model

def preprocess_input(x, **kwargs):
  #Numpy 배열을 이미지 배치로 전처리
  return imagenet_utils.preprocess_input(x, mode = 'tf', **kwargs)